1. Write data to delta lake (managed table)
2. Write data to delta lake (external table)
3. Read data from delta lake (Table)
4. Read data from delta lake (File)

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS f1_demo
LOCATION '/mnt/formula1dl/demo'

In [0]:
results_df = spark.read \
.option("inferSchema", True) \
.json("/mnt/formula1dl/raw/2021-03-28/results.json")

In [0]:
results_df.write.format("delta").mode("overwrite").saveAsTable("f1_demo.results_managed")

In [0]:
%sql
SELECT * FROM f1_demo.results_managed;

In [0]:
results_df.write.format("delta").mode("overwrite").save("/mnt/formula1dl/demo/results_external")

In [0]:
%sql
CREATE TABLE f1_demo.results_external
USING DELTA
LOCATION '/mnt/formula1dl/demo/results_external'

In [0]:
%sql
SELECT * FROM f1_demo.results_external

In [0]:
results_external_df = spark.read.format("delta").load("/mnt/formula1dl/demo/results_external")

In [0]:
display(results_external_df)

In [0]:
results_df.write.format("delta").mode("overwrite").partitionBy("constructorId").saveAsTable("f1_demo.results_partitioned")

In [0]:
%sql
SHOW PARTITIONS f1_demo.results_partitioned

1. Update Delta Table
2. Delete From Delta Table

In [0]:
%sql
SELECT * FROM f1_demo.results_managed;

In [0]:
%sql
UPDATE f1_demo.results_managed
  SET points = 11 - position
WHERE position <= 10

In [0]:
%sql
SELECT * FROM f1_demo.results_managed;

In [0]:
from delta.tables import DeltaTable

deltaTable = DeltaTable.forPath(spark, "/mnt/formula1dl/demo/results_managed")

deltaTable.update("position <= 10", { "points": "21 - position" } ) 

In [0]:
%sql
SELECT * FROM f1_demo.results_managed;

In [0]:
%sql
DELETE FROM f1_demo.results_managed
WHERE position > 10;

In [0]:
%sql
SELECT * FROM f1_demo.results_managed;

In [0]:
from delta.tables import DeltaTable

deltaTable = DeltaTable.forPath(spark, "/mnt/formula1dl/demo/results_managed")

deltaTable.delete("points = 0") 

In [0]:
%sql
SELECT * FROM f1_demo.results_managed;

Upsert using merge

In [0]:
drivers_day1_df = spark.read \
.option("inferSchema", True) \
.json("/mnt/formula1dl/raw/2021-03-28/drivers.json") \
.filter("driverId <= 10") \
.select("driverId", "dob", "name.forename", "name.surname")

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-3624191377186925> in <module> 
 ----> 1 drivers_day1_df = spark . read \ 
 2 . option ( "inferSchema" , True ) \ 
 3 . json ( "/mnt/formula1dl/raw/2021-03-28/drivers.json" ) \ 
 4 . filter ( "driverId <= 10" ) \ 
 5 . select ( "driverId" , "dob" , "name.forename" , "name.surname" ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in json (self, path, schema, primitivesAsString, prefersDecimal, allowComments, allowUnquotedFieldNames, allowSingleQuotes, allowNumericLeadingZero, allowBackslashEscapingAnyCharacter, mode, columnNameOfCorruptRecord, dateFormat, timestampFormat, multiLine, allowUnquotedControlChars, lineSep, samplingRatio, dropFieldIfAllNull, encoding, locale, pathGlobFilter, recursiveFileLookup, allowNonNumericNumbers, modifiedBefore, modifiedAfter) 
 227 path = [ path ] 
 228 if type ( path ) == list : 
 --> 229 return self . _df ( self . _jreader . json ( self . _spark . _sc . _jvm . PythonUtils . toSeq ( path ) ) ) 
 230 elif isinstance ( path , RDD ) : 
 231 def func ( iterator ) : 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : Path does not exist: dbfs:/mnt/formula1dl/raw/2021-03-28/drivers.json

In [0]:
display(drivers_day1_df)

In [0]:
drivers_day1_df.createOrReplaceTempView("drivers_day1")

In [0]:
from pyspark.sql.functions import upper

drivers_day2_df = spark.read \
.option("inferSchema", True) \
.json("/mnt/formula1dl/raw/2021-03-28/drivers.json") \
.filter("driverId BETWEEN 6 AND 15") \
.select("driverId", "dob", upper("name.forename").alias("forename"), upper("name.surname").alias("surname"))

In [0]:
drivers_day2_df.createOrReplaceTempView("drivers_day2")

In [0]:
display(drivers_day2_df)

In [0]:
from pyspark.sql.functions import upper

drivers_day3_df = spark.read \
.option("inferSchema", True) \
.json("/mnt/formula1dl/raw/2021-03-28/drivers.json") \
.filter("driverId BETWEEN 1 AND 5 OR driverId BETWEEN 16 AND 20") \
.select("driverId", "dob", upper("name.forename").alias("forename"), upper("name.surname").alias("surname"))

In [0]:
%sql
CREATE TABLE IF NOT EXISTS f1_demo.drivers_merge (
driverId INT,
dob DATE,
forename STRING, 
surname STRING,
createdDate DATE, 
updatedDate DATE
)
USING DELTA

Day1

In [0]:
%sql
MERGE INTO f1_demo.drivers_merge tgt
USING drivers_day1 upd
ON tgt.driverId = upd.driverId
WHEN MATCHED THEN
  UPDATE SET tgt.dob = upd.dob,
             tgt.forename = upd.forename,
             tgt.surname = upd.surname,
             tgt.updatedDate = current_timestamp
WHEN NOT MATCHED
  THEN INSERT (driverId, dob, forename,surname,createdDate ) VALUES (driverId, dob, forename,surname, current_timestamp)

In [0]:
%sql SELECT * FROM f1_demo.drivers_merge;

Day 2

In [0]:
%sql
MERGE INTO f1_demo.drivers_merge tgt
USING drivers_day2 upd
ON tgt.driverId = upd.driverId
WHEN MATCHED THEN
  UPDATE SET tgt.dob = upd.dob,
             tgt.forename = upd.forename,
             tgt.surname = upd.surname,
             tgt.updatedDate = current_timestamp
WHEN NOT MATCHED
  THEN INSERT (driverId, dob, forename,surname,createdDate ) VALUES (driverId, dob, forename,surname, current_timestamp)

In [0]:
%sql SELECT * FROM f1_demo.drivers_merge;

Day 3

In [0]:
from pyspark.sql.functions import current_timestamp
from delta.tables import DeltaTable

deltaTable = DeltaTable.forPath(spark, "/mnt/formula1dl/demo/drivers_merge")

deltaTable.alias("tgt").merge(
    drivers_day3_df.alias("upd"),
    "tgt.driverId = upd.driverId") \
  .whenMatchedUpdate(set = { "dob" : "upd.dob", "forename" : "upd.forename", "surname" : "upd.surname", "updatedDate": "current_timestamp()" } ) \
  .whenNotMatchedInsert(values =
    {
      "driverId": "upd.driverId",
      "dob": "upd.dob",
      "forename" : "upd.forename", 
      "surname" : "upd.surname", 
      "createdDate": "current_timestamp()"
    }
  ) \
  .execute()

In [0]:
%sql SELECT * FROM f1_demo.drivers_merge;

1. History & Versioning
2. Time Travel
3. Vaccum

In [0]:
%sql
DESC HISTORY f1_demo.drivers_merge

In [0]:
%sql
SELECT * FROM f1_demo.drivers_merge VERSION AS OF 2;

In [0]:
%sql
SELECT * FROM f1_demo.drivers_merge TIMESTAMP AS OF '2021-06-23T15:40:33.000+0000';

In [0]:
df = spark.read.format("delta").option("timestampAsOf", '2021-06-23T15:40:33.000+0000').load("/mnt/formula1dl/demo/drivers_merge")

In [0]:
display(df)

In [0]:
%sql
VACUUM f1_demo.drivers_merge

In [0]:
%sql
SELECT * FROM f1_demo.drivers_merge TIMESTAMP AS OF '2021-06-23T15:40:33.000+0000';

In [0]:
%sql
SET spark.databricks.delta.retentionDurationCheck.enabled = false;
VACUUM f1_demo.drivers_merge RETAIN 0 HOURS

In [0]:
%sql
SELECT * FROM f1_demo.drivers_merge TIMESTAMP AS OF '2021-06-23T15:40:33.000+0000';

In [0]:
%sql
SELECT * FROM f1_demo.drivers_merge

In [0]:
%sql
DESC HISTORY f1_demo.drivers_merge;

In [0]:
%sql
DELETE FROM f1_demo.drivers_merge WHERE driverId = 1;

In [0]:
%sql 
SELECT * FROM f1_demo.drivers_merge VERSION AS OF 3;

In [0]:
%sql
MERGE INTO f1_demo.drivers_merge tgt
USING f1_demo.drivers_merge VERSION AS OF 3 src
   ON (tgt.driverId = src.driverId)
WHEN NOT MATCHED THEN
   INSERT *

In [0]:
%sql DESC HISTORY f1_demo.drivers_merge

In [0]:
%sql
SELECT * FROM f1_demo.drivers_merge

Transaction Logs

In [0]:
%sql
CREATE TABLE IF NOT EXISTS f1_demo.drivers_txn (
driverId INT,
dob DATE,
forename STRING, 
surname STRING,
createdDate DATE, 
updatedDate DATE
)
USING DELTA

In [0]:
%sql
DESC HISTORY f1_demo.drivers_txn

In [0]:
%sql
INSERT INTO f1_demo.drivers_txn
SELECT * FROM f1_demo.drivers_merge
WHERE driverId = 1;

In [0]:
%sql
DESC HISTORY f1_demo.drivers_txn

In [0]:
%sql
INSERT INTO f1_demo.drivers_txn
SELECT * FROM f1_demo.drivers_merge
WHERE driverId = 2;

In [0]:
%sql
DELETE FROM  f1_demo.drivers_txn
WHERE driverId = 1;

In [0]:
for driver_id in range(3, 20):
  spark.sql(f"""INSERT INTO f1_demo.drivers_txn
                SELECT * FROM f1_demo.drivers_merge
                WHERE driverId = {driver_id}""")

In [0]:
%sql
INSERT INTO f1_demo.drivers_txn
SELECT * FROM f1_demo.drivers_merge;

Convert Parquet to Delta

In [0]:
%sql
CREATE TABLE IF NOT EXISTS f1_demo.drivers_convert_to_delta (
driverId INT,
dob DATE,
forename STRING, 
surname STRING,
createdDate DATE, 
updatedDate DATE
)
USING PARQUET

In [0]:
%sql
INSERT INTO f1_demo.drivers_convert_to_delta
SELECT * FROM f1_demo.drivers_merge

In [0]:
%sql
CONVERT TO DELTA f1_demo.drivers_convert_to_delta

In [0]:
df = spark.table("f1_demo.drivers_convert_to_delta")

In [0]:
df.write.format("parquet").save("/mnt/formula1dl/demo/drivers_convert_to_delta_new")

In [0]:
%sql
CONVERT TO DELTA parquet.`/mnt/formula1dl/demo/drivers_convert_to_delta_new`